# BingSearch using Kernel 

This notebook explains how to integrate Bing Search with the Semantic Kernel  to get the latest information from the internet.

To use Bing Search you simply need a Bing Search API key. You can get the API key by creating a [Bing Search resource](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/create-bing-search-service-resource) in Azure. 

To learn more read the following [documenation](https://learn.microsoft.com/en-us/bing/search-apis/bing-web-search/overview)


Prepare a semantic kernel instance first, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"
#r "nuget: Microsoft.SemanticKernel.Skills.Web, 0.17.230626.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;
using Microsoft.SemanticKernel.TemplateEngine;
var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAITextCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Add the following namespaces to use the Bing Search connector.

In [3]:
using Microsoft.SemanticKernel.Skills.Web;
using Microsoft.SemanticKernel.Skills.Web.Bing;

Below are some examples on how `WebSearchEngineSkill` can be use in SK . Just uncomment the example to use the .

In [15]:
// Load Bing skill
//Uncomment the Example 
using (var bingConnector = new BingConnector("BING_KEY")
{
kernel.ImportSkill(new WebSearchEngineSkill(bingConnector), "bing");
//First run the example sample given below and uncomment the await the  following statements 
//await Example1Async(kernel);
//await Example2Async(kernel);
}

Example 1
What is quantum tunnelling
----
In physics, quantum tunnelling, barrier penetration, or simply tunnelling is a quantum mechanical phenomenon in which an object such as an electron or atom passes through a potential energy barrier that, according to classical mechanics, the object does not have sufficient energy to enter or surmount.

Example 2
[1] Elon Musk (@elonmusk) is the owner of Twitter and is the most-followed person on Twitter, with over 144 million followers. Former U.S. President Barack Obama (@BarackObama) is the second most-followed person on Twitter, with over 132 million followers.


In [13]:
private static async Task Example1Async(IKernel kernel)
{
        Console.WriteLine("Example 1");

        // Run 
        var question = "What is quantum tunnelling";
        var bingResult = await kernel.Func("bing", "search").InvokeAsync(question);

        Console.WriteLine(question);
        Console.WriteLine("----");
        Console.WriteLine(bingResult);
        Console.WriteLine();

        /* OUTPUT:

            What is quantum tunnerling
            ----
            In physics, quantum tunnelling, barrier penetration, or simply tunnelling is a quantum mechanical
            phenomenon in which an object such as an electron or atom passes through a potential energy 
            barrier that, according to classical mechanics, the object does not have sufficient energy to
            enter or surmount.
            
       */
}

In [14]:
private static async Task Example2Async(IKernel kernel)
{
    Console.WriteLine("Example 2");
    //Use the following function in VS only
    string question = await InteractiveKernel.GetInputAsync("Please aSk your question"); 
    //Console.ReadLine();
    var bingResult = await kernel.Func("bing", "search").InvokeAsync(question);

        Console.WriteLine(bingResult);
}